# Agenda  <a name="backtotop"></a>
---

1. [Nested Dictionary from CSV](#NestedDict)


## Nested Dictionary from CSV <a name="NestedDict">
</a>


[back to top](#backtotop)

In [ ]:
import nltk as nltk
import csv
import re
from functools import reduce
from collections import Counter

class Tree(dict):
    def __missing__(self, key):
        value = self[key] = type(self)()
        return value

def nested_dict_get(nested_dict, keys):
    return reduce(lambda d, k: d[k], keys, nested_dict)

def nested_dict_set(nested_dict, keys, value):
    nested_dict_get(nested_dict, keys[:-1])[keys[-1]] = value

def nested_dict_update_count(nested_dict, keys):
    if nested_dict_get(nested_dict, keys[:-1]):  # update existing Counter
        nested_dict_get(nested_dict, keys[:-1]).update([keys[-1]])
    else:                                        # create a new  Counter
        nested_dict_set(nested_dict, keys[:-1], Counter([keys[-1]]))



class CreateDIYdict:
    @staticmethod
    def counter2list(counter):
        return list(counter.elements())

    @staticmethod
    def nest(x):
        return nltk.defaultdict(lambda: x)

    def __init__(self,path,filename,skip_badlines=False,encoding='utf-8-sig',delimiter=','):
        InfoDict=nltk.defaultdict(list)
        with open(path + filename, encoding=encoding) as csv_data:
            reader = csv.reader(csv_data,delimiter=delimiter)
            headers = next(reader, None)
        Headerlist=[(head,headers.index(head)) for head in headers]
        with open(path + filename, encoding=encoding) as csv_data:
            File = csv.reader(csv_data,delimiter=delimiter)
            next(File)
            for line in File:
                if skip_badlines:
                    try:
                        if len(line)>=len(headers):
                            for head,index in Headerlist:
                                InfoDict[head].append(line[index])
                        else:
                            pass
                    except:
                        pass
                else:
                    for head, index in Headerlist:
                        InfoDict[head].append(line[index])
        self.Headerlist=Headerlist
        self.AllInfo=dict(InfoDict)


# valuespatterns is dict or list
    def DIY_dict(self,values=None,valuespatterns=None,convert_to=None,only_keep_first_match=True):
        AllInfo=self.AllInfo
        if values is None:
            values=[header for (header,index) in self.Headerlist]
        Infolist=[AllInfo[value] for value in values]
        Range=len(Infolist[0])
        DIYdict = Tree()
        for n in range(Range):
            Keys = []
            for i,header in enumerate(values):
                try:
                    if type(valuespatterns) is dict:
                        pattern=valuespatterns[header]
                    else:
                        pattern=valuespatterns[i]
                    Find=re.findall(pattern,Infolist[i][n])
                    if only_keep_first_match:
                        Find.append('NOT FIND')
                        Keys.append(Find[0])
                    else:
                        if Find:
                            Keys.append(tuple(Find))
                        else:
                            Find.append('NOT FIND')
                            Keys.append(tuple(Find))
                except:
                    Keys.append(Infolist[i][n])
            nested_dict_update_count(DIYdict,Keys)
        if convert_to is None:
            return DIYdict
        else:
            DIYdict=dict((k,convert_to(CreateDIYdict.counter2list(v))) for k,v in DIYdict.items())
            return DIYdict
        

        
        
        
path=''
OBJ=CreateDIYdict(path,'..csv',skip_badlines=True,encoding='Windows-1252')
BonfireCodeDict=OBJ.DIY_dict(['column1','column2'],convert_to=set)